## 1. Import Libraries and Load Data
In this section, we load the census tract shapefile and air quality monitor CSV data. 
We then convert the monitor CSV to a GeoDataFrame and check that both datasets are loaded correctly.

In [2]:
# Core numerical and data manipulation libraries
import numpy as np
import pandas as pd
import collections
from kneed import KneeLocator

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import seaborn as sns
import folium

# Geospatial libraries
import geopandas as gpd
from shapely.geometry import Point, LineString

# Machine learning for spatial clustering
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from scipy.stats import boxcox

# Spatial statistical analysis
import libpysal
from esda import Moran, Moran_Local

# Network analysis and OSM data
import osmnx as ox
import networkx as nx

from scripts.data_loader import load_census_tracts, load_monitors

import warnings
warnings.filterwarnings("ignore", message="The weights matrix is not fully connected")
warnings.filterwarnings("ignore", message="The weights matrix is not symmetric")
warnings.filterwarnings("ignore", message="The weights matrix is not square")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Set the plotting style
sns.set_style('whitegrid')
# Set the color palette
sns.set_palette('deep')
# Set the figure size for plots
plt.rcParams['figure.figsize'] = (10, 6)
# Set the font size for plots
plt.rcParams['font.size'] = 12
# Set the color palette for plots
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=sns.color_palette())
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.edgecolor'] = 'black'
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.titlesize'] = 'large'
plt.rcParams['axes.labelsize'] = 'large'
# plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.labelcolor'] = 'black'
plt.rcParams['axes.titlecolor'] = 'black'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['axes.titlepad'] = 10

# Load Census Tract shapefile
tracts_gdf = load_census_tracts("data/tl_2024_06_tract.zip")

# Load Air Quality Monitor CSV
monitors_gdf = load_monitors("data/annual_conc_by_monitor_2024.csv")

# Create GeoDataFrame for monitors, using Longitude and Latitude columns
monitors_gdf = gpd.GeoDataFrame(
    monitors_df, 
    geometry=gpd.points_from_xy(monitors_df['Longitude'], monitors_df['Latitude']),
    crs="EPSG:4326"
)

# Add a unique identifier for monitors if not present
monitors_gdf.reset_index(inplace=True)
monitors_gdf.rename(columns={'index': 'monitor_id'}, inplace=True)

print("Shape of tracts:", tracts.shape)
print("Shape of monitors:", monitors_gdf.shape)

ModuleNotFoundError: No module named 'scripts'

## 2. Data Inspection: Geometry Complexity and Area Statistics
We inspect the census tract dataset to determine:
- Total number of features
- Geometry complexity (number of vertices per feature)
- Area statistics (min, max, average area in square kilometers)

### Basic Info

In [ ]:
# Print high-level info about the GeoDataFrames
print("TRACTS INFO:")
tracts.info()
print("\nMONITORS INFO:")
monitors_gdf.info()

# Peek at a few rows
print("\nTRACTS HEAD:")
display(tracts.head(3))
print("\nMONITORS HEAD:")
display(monitors_gdf.head(3))

# If you want descriptive stats for numeric columns:
print("\nDescriptive Stats for MONITORS:")
display(monitors_gdf.describe(include='all'))

### Missing Values

In [ ]:
# Count missing values in each column
print("Missing values in tracts:")
print(tracts.isna().sum())

print("\nMissing values in monitors_gdf:")
print(monitors_gdf.isna().sum())

### Pollutant Details

In [ ]:
target_pollutant = "Ozone"  # or "PM2.5 - Local Conditions"
subset = monitors_gdf[monitors_gdf["Parameter Name"] == target_pollutant]
print(f"\nNumber of rows for {target_pollutant}:", len(subset))
display(subset.describe())

In [ ]:
sns.histplot(data=subset, x="Arithmetic Mean", kde=True)
plt.title(f"Distribution of {target_pollutant} Arithmetic Mean")
plt.show()

### Coordinate Reference Systems (CRS)

In [ ]:
# Convert tracts to EPSG:4326 (WGS84) for consistency in some operations
tracts_filtered_wgs = tracts.to_crs(epsg=4326)

# 1. Number of features
num_features = len(tracts_filtered_wgs)
print("Number of features in tracts_filtered_wgs:", num_features)

# 2. Geometry Complexity
def count_vertices(geom):
    """Count number of vertices in a geometry (for Polygon or MultiPolygon)."""
    if geom.geom_type == 'Polygon':
        return len(geom.exterior.coords)
    elif geom.geom_type == 'MultiPolygon':
        return sum(len(part.exterior.coords) for part in geom.geoms)
    else:
        return 0

tracts_filtered_wgs['num_vertices'] = tracts_filtered_wgs.geometry.apply(count_vertices)
avg_vertices = tracts_filtered_wgs['num_vertices'].mean()
min_vertices = tracts_filtered_wgs['num_vertices'].min()
max_vertices = tracts_filtered_wgs['num_vertices'].max()
print("\nGeometry Complexity (vertices per feature):")
print(f" - Average vertices: {avg_vertices:.2f}")
print(f" - Minimum vertices: {min_vertices}")
print(f" - Maximum vertices: {max_vertices}")

# 3. Area Statistics
# Reproject tracts to EPSG:3310 (California Albers Equal Area) for accurate area calculation
tracts_proj = tracts.to_crs(epsg=3310)
tracts_filtered_wgs['area_sq_km'] = tracts_proj.geometry.area / 1e6  # area in sq km

min_area = tracts_filtered_wgs['area_sq_km'].min()
max_area = tracts_filtered_wgs['area_sq_km'].max()
avg_area = tracts_filtered_wgs['area_sq_km'].mean()
print("\nCensus Tract Area (sq km) Statistics:")
print(f" - Minimum area: {min_area:.2f} sq km")
print(f" - Maximum area: {max_area:.2f} sq km")
print(f" - Average area: {avg_area:.2f} sq km")

print("\nMonitors CRS:", monitors_gdf.crs)
print("tracts_filtered_wgs CRS:", tracts_filtered_wgs.crs)

# Inspect a few rows of each dataset
print("Number of census tracts loaded:", len(tracts))
print("Number of monitors loaded:", len(monitors_gdf))

## Filtering and Cleaning

### Invalid Geometry or Duplicates

In [ ]:
# Drop any rows with missing geometry in monitors
monitors_gdf = monitors_gdf[monitors_gdf.geometry.notna()].copy()

# Drop duplicates if needed (e.g., duplicates in monitor ID + geometry)
monitors_gdf.drop_duplicates(subset=["monitor_id", "Latitude", "Longitude"], inplace=True)

### Out-of-Range Coordinates

In [ ]:
mask_valid_coords = (
    (monitors_gdf.geometry.y >= -90) & (monitors_gdf.geometry.y <= 90) &
    (monitors_gdf.geometry.x >= -180) & (monitors_gdf.geometry.x <= 180)
)
monitors_gdf = monitors_gdf[mask_valid_coords].copy()

## Column Selection and Renaming

In [ ]:
# Example: Keep only certain columns
cols_to_keep = [
    "monitor_id", "State Code", "County Code", "Site Num", "Parameter Name",
    "Year", "Units of Measure", "Arithmetic Mean", "Latitude", "Longitude",
    "geometry"
]
monitors_gdf = monitors_gdf[cols_to_keep].copy()

# Optional: Rename columns for clarity
monitors_gdf.rename(columns={
    "Arithmetic Mean": "value_mean",
    "Parameter Name": "pollutant_name"
}, inplace=True)

## Pollutant-Specific Preprocessing

In [ ]:
# Filter for Ozone monitors
ozone_gdf = monitors_gdf[monitors_gdf["pollutant_name"] == "Ozone"].copy()
ozone_gdf.info()

# If you want a certain year or range of years, filter further
ozone_gdf = ozone_gdf[ozone_gdf["Year"] == 2024]  

### Handling Missing or Zero Values

In [ ]:
# Drop or fill missing values
ozone_gdf = ozone_gdf[ozone_gdf["value_mean"].notna()].copy()

# If you suspect zeros are invalid, filter them out:
ozero_mask = ozone_gdf["value_mean"] > 0.0
ozone_gdf = ozone_gdf[ozero_mask].copy()

### Removing Outliers

In [ ]:
# Remove top/bottom 1% of values
low_cut = ozone_gdf["value_mean"].quantile(0.01)
high_cut = ozone_gdf["value_mean"].quantile(0.99)
mask_outliers = (ozone_gdf["value_mean"] >= low_cut) & (ozone_gdf["value_mean"] <= high_cut)
ozone_gdf = ozone_gdf[mask_outliers].copy()

### Transformations

In [ ]:
# Box-Cox transform
ozone_boxcox, lambda_ = boxcox(ozone_gdf["value_mean"])
ozone_gdf["value_boxcox"] = ozone_boxcox
print("Box-Cox lambda:", lambda_)

## Spatial Join & Mapping

In this step, we integrate the air quality monitor data with census tract polygons and visualize the spatial distribution of pollution. We will:
- Spatially join air quality monitor points to their containing census tracts.
- Aggregate pollutant metrics per tract, using ozone (O₃).
- Visualize the data with a static choropleth map (using Matplotlib) and an interactive map (using Folium), and save the interactive map as an HTML file.

### Spatial Join of Monitors to Census Tracts

In [ ]:
# Ensure both GeoDataFrames have the same CRS (project if necessary)
# Make sure 'ozone_gdf' uses the same CRS as 'tracts'
ozone_gdf = ozone_gdf.to_crs(tracts.crs)

# Perform spatial join: assign each monitor the attributes of the tract it falls in
# Spatially join the Ozone monitors to census tracts
ozone_in_tracts = gpd.sjoin(
    ozone_gdf,      # your preprocessed Ozone data
    tracts,         # the census tract polygons
    how='inner',    # only keep monitors that fall inside a tract
    predicate='intersects'
)

print(f"Joined Ozone records: {len(ozone_in_tracts)}")
display(ozone_in_tracts.head(3))

### Aggregating Pollutant Metrics per Tract

In [ ]:
grouped = ozone_in_tracts.groupby('GEOID')['value_mean'].mean().reset_index()

# Rename the aggregated column to 'Ozone' for clarity
grouped.rename(columns={'value_mean': 'Ozone'}, inplace=True)

print("Aggregated Ozone data (first few rows):")
display(grouped.head(5))

In [ ]:
# Make a copy of tracts for clarity
tracts_ozone = tracts.copy()

# Merge on the 'GEOID' column
tracts_ozone = tracts_ozone.merge(grouped, on='GEOID', how='left')

# Check how many tracts got Ozone data
num_ozone = tracts_ozone['Ozone'].notna().sum()
print(f"Tracts with Ozone data: {num_ozone} / {len(tracts_ozone)}")

### Static Map of Pollutant Levels by Tract

In [ ]:
# Ensure 'tracts_ozone' is in EPSG:4326 if you want lat/lon plotting
tracts_ozone = tracts_ozone.to_crs(epsg=4326)

fig, ax = plt.subplots(figsize=(10, 8))

# Plot tracts colored by 'Ozone' column
tracts_ozone.plot(
    column='Ozone',
    cmap='OrRd',
    legend=True,
    ax=ax,
    edgecolor='black',
    linewidth=0.5,
    alpha=0.8,
    missing_kwds={
        "color": "lightgrey",
        "edgecolor": "black",
        "hatch": "",
        "label": "No Data"
    }
)

ax.set_title("Census Tracts by Mean Ozone Concentration", fontsize=14)
ax.set_xlabel("Longitude", fontsize=12)
ax.set_ylabel("Latitude", fontsize=12)

# Force the axes to show the full extent of the data
minx, miny, maxx, maxy = tracts_ozone.total_bounds
ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

# Create a custom legend entry for missing data
missing_patch = mpatches.Patch(facecolor="lightgrey", edgecolor="black", label="No Data")
ax.legend(handles=[missing_patch], loc='lower left', fontsize=12)

plt.tight_layout()
plt.show()

### Interactive Map with Folium

In [ ]:
# Center the map roughly over California
m = folium.Map(location=[36.77, -119.42], zoom_start=6)

# Convert 'tracts_ozone' to GeoJSON for Folium
# We'll drop rows with no geometry or Ozone for clarity
tracts_for_folium = tracts_ozone.dropna(subset=['Ozone', 'geometry']).copy()

# Add the choropleth
folium.Choropleth(
    geo_data=tracts_for_folium,
    name="Ozone",
    data=tracts_for_folium,
    columns=["GEOID", "Ozone"],
    key_on="feature.properties.GEOID",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Mean Ozone Concentration"
).add_to(m)

folium.LayerControl().add_to(m)
m.save("maps/ozone_map.html")
m

## Spatial Clustering of Air Quality Monitors

### Determining the Optimal Epsilon (ε) with k-Distance Graph

In [ ]:
# 1) Ensure both Ozone data and tracts share the same CRS
ozone_gdf = ozone_gdf.to_crs(tracts.crs)

# 2) Spatially join Ozone monitors with census tracts
ozone_in_tracts = gpd.sjoin(
    ozone_gdf, 
    tracts, 
    how='inner', 
    predicate='intersects'
)

print(f"Number of Ozone monitors after join: {len(ozone_in_tracts)}")
display(ozone_in_tracts.head(3))

# Option A: Use the original 'value_mean' for Ozone
# ozone_in_tracts['Ozone'] = ozone_in_tracts['value_mean']

# Option B: Use your log transform or Box-Cox transform instead
# ozone_in_tracts['Ozone'] = ozone_in_tracts['value_log']
ozone_in_tracts['Ozone'] = ozone_in_tracts['value_boxcox']

# Drop any rows with missing Ozone
ozone_in_tracts = ozone_in_tracts[ozone_in_tracts['Ozone'].notna()].copy()

# Extract coordinate arrays (make sure the geometry is still in a projected or lat/lon CRS)
coords = np.array(list(zip(ozone_in_tracts.geometry.x, ozone_in_tracts.geometry.y)))

# Combine coordinates + Ozone values into a single feature matrix X
ozone_vals = ozone_in_tracts['Ozone'].values
X = np.column_stack([coords, ozone_vals])

print("Feature matrix shape:", X.shape)

k = 5  # If min_samples=6, then k=5 for the k-distance
nbrs = NearestNeighbors(n_neighbors=k).fit(X)
distances, indices = nbrs.kneighbors(X)
kth_distances = np.sort(distances[:, k-1])

plt.figure(figsize=(6,4))
plt.plot(kth_distances)
plt.title(f"{k}-distance graph for DBSCAN")
plt.xlabel("Point (sorted by distance to 5th neighbor)")
plt.ylabel(f"Distance to {k}th nearest neighbor")
plt.show()

### Applying DBSCAN Clustering

In [ ]:
# Choose epsilon based on the k-distance elbow, e.g., eps=0.5
eps_opt = 0.52
min_pts = 6

db = DBSCAN(eps=eps_opt, min_samples=min_pts)
labels = db.fit_predict(X)

ozone_in_tracts['cluster'] = labels
num_clusters = len(set(labels) - {-1})
noise_points = list(labels).count(-1)

print(f"DBSCAN identified {num_clusters} clusters")
print(f"Noise points (outliers): {noise_points}")

cluster_counts = pd.Series(labels).value_counts().sort_index()
print("Cluster sizes:", cluster_counts.to_dict())

### Mapping the Clusters

In [ ]:
# Define a map center (e.g., approximate center of California)
latitude_center = 36.77  
longitude_center = -119.42

m_clusters = folium.Map(location=[latitude_center, longitude_center], zoom_start=6)

# (Optional) Add tract boundaries in light gray
folium.GeoJson(
    tracts.to_crs(epsg=4326).geometry,  # ensure tracts are in lat/lon
    name="Tracts",
    style_function=lambda x: {'fillColor': 'transparent', 'color': 'black', 'weight': 0.5}
).add_to(m_clusters)

# Define colors for clusters
num_clusters = len(set(labels) - {-1})
colormap = cm.get_cmap('tab10', num_clusters)  # discrete colormap
cluster_colors = {
    i: f"#{int(colormap(i)[0]*255):02x}{int(colormap(i)[1]*255):02x}{int(colormap(i)[2]*255):02x}"
    for i in range(num_clusters)
}
# Noise color
cluster_colors[-1] = "gray"

# Add monitor points to the map
for idx, row in ozone_in_tracts.iterrows():
    cl = row['cluster']
    color = cluster_colors.get(cl, "gray")
    # For the popup, we show the cluster label and Ozone concentration
    popup_text = (
        f"Monitor ID: {row['monitor_id']}<br>"
        f"Cluster: {cl}<br>"
        f"Ozone: {row['Ozone']:.4f}"
    )
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.8,
        popup=popup_text
    ).add_to(m_clusters)

folium.LayerControl().add_to(m_clusters)

# Save or display
m_clusters.save("maps/ozone_clusters_map.html")
m_clusters

## Spatial Autocorrelation Analysis

### Aggregate Ozone Monitors to Tracts

In [ ]:
# ------------------------------------------------------------------------------
# 1) Ensure 'ozone_gdf' (preprocessed Ozone monitors) and 'tracts' share the same CRS
# ------------------------------------------------------------------------------
ozone_gdf = ozone_gdf.to_crs(tracts.crs)

# ------------------------------------------------------------------------------
# 2) Spatial join: attach tract info to each Ozone monitor
# ------------------------------------------------------------------------------
ozone_in_tracts = gpd.sjoin(
    ozone_gdf,    # preprocessed Ozone monitors
    tracts,       # census tracts
    how='inner',
    predicate='intersects'
)

print(f"Joined Ozone records: {len(ozone_in_tracts)}")
display(ozone_in_tracts.head(3))

# ------------------------------------------------------------------------------
# 3) Aggregate Ozone by tract (mean Ozone per GEOID)
# ------------------------------------------------------------------------------
grouped = ozone_in_tracts.groupby('GEOID')['value_mean'].mean().reset_index()
grouped.rename(columns={'value_mean': 'Ozone'}, inplace=True)

print("Aggregated Ozone data (first few rows):")
display(grouped.head(5))

# ------------------------------------------------------------------------------
# 4) Merge aggregated Ozone data back to the census tracts
# ------------------------------------------------------------------------------
tracts_gdf = tracts.copy()  # keep original tracts safe
tracts_gdf = tracts_gdf.merge(grouped, on='GEOID', how='left')

# Print stats about the Ozone column
num_ozone = tracts_gdf['Ozone'].notna().sum()
print(f"Tracts with Ozone data: {num_ozone} / {len(tracts_gdf)}")
print("Min Ozone:", tracts_gdf['Ozone'].min(skipna=True))
print("Max Ozone:", tracts_gdf['Ozone'].max(skipna=True))
print("Mean Ozone:", tracts_gdf['Ozone'].mean(skipna=True))

### Construct Spatial Weights & Remove Islands

In [ ]:
# ------------------------------------------------------------------------------
# 5) Construct spatial weights (Queen contiguity) & row-standardize
# ------------------------------------------------------------------------------
w = libpysal.weights.Queen.from_dataframe(tracts_gdf, use_index=False)
w.transform = 'R'

# Identify islands (tracts with no neighbors) and remove them
islands = w.islands
if islands:
    print("Removing island tracts with indices:", islands)
    tracts_gdf = tracts_gdf.drop(index=islands).copy()
    # Recompute weights on the filtered data
    w = libpysal.weights.Queen.from_dataframe(tracts_gdf, use_index=False)
    w.transform = 'R'

# ------------------------------------------------------------------------------
# 6) Check connected components
# ------------------------------------------------------------------------------
components = w.component_labels
component_counts = collections.Counter(components)
print("Component counts:", component_counts)

# Identify the largest connected component
largest_component = component_counts.most_common(1)[0][0]
print("Largest component label:", largest_component)

# Create a mask for tracts belonging to the largest component
mask = np.array(components) == largest_component
tracts_gdf_largest = tracts_gdf.iloc[mask].copy()

# Recompute spatial weights for the largest component
w = libpysal.weights.Queen.from_dataframe(tracts_gdf_largest, use_index=False)
w.transform = 'R'

### Global Moran’s I for Ozone

In [ ]:
# ------------------------------------------------------------------------------
# 7) Extract Ozone values & compute Global Moran's I
# ------------------------------------------------------------------------------
y = tracts_gdf_largest['Ozone'].values

print("After filtering to the largest component:")
print("Min Ozone:", np.nanmin(y))
print("Max Ozone:", np.nanmax(y))
print("Mean Ozone:", np.nanmean(y))

moran = Moran(y, w)
print(f"Global Moran's I: {moran.I:.3f}")
print(f"P-value: {moran.p_sim:.3f} (two-sided)")

### Local Moran’s I (LISA) and Cluster Mapping

In [ ]:
# ------------------------------------------------------------------------------
# 8) Local Moran's I
# ------------------------------------------------------------------------------
lisa = Moran_Local(y, w)

local_I = lisa.Is     # local Moran's I values
p_vals = lisa.p_sim   # p-values for each local I
signif = p_vals < 0.05  # significance at 95%

# ------------------------------------------------------------------------------
# 9) Classify each tract by cluster type
# ------------------------------------------------------------------------------
clusters = []
for i, val in enumerate(y):
    if not signif[i]:
        clusters.append('Not significant')
    else:
        # Compare value and its spatial lag
        if val > y.mean() and lisa.y_z[i] > 0:
            clusters.append('High-High')
        elif val < y.mean() and lisa.y_z[i] < 0:
            clusters.append('Low-Low')
        elif val > y.mean() and lisa.y_z[i] < 0:
            clusters.append('High-Low')
        elif val < y.mean() and lisa.y_z[i] > 0:
            clusters.append('Low-High')
        else:
            clusters.append('Not significant')

tracts_gdf_largest['LISA_cluster'] = clusters

print("Local cluster counts:")
print(pd.Series(clusters).value_counts())

### LISA Cluster Significance Map

In [ ]:
# ------------------------------------------------------------------------------
# 10) Plot the LISA cluster map
# ------------------------------------------------------------------------------
tracts_proj = tracts_gdf_largest.to_crs(epsg=3857)

color_map = {
    'High-High': 'red',
    'Low-Low': 'blue',
    'High-Low': 'orange',
    'Low-High': 'lightblue',
    'Not significant': 'lightgray'
}

fig, ax = plt.subplots(figsize=(8, 6))

# Plot all tracts in light gray
tracts_proj.plot(color='lightgray', ax=ax, edgecolor='white')

# Collect handles for a custom legend
legend_handles = []

# Loop over cluster types
for ctype, color in color_map.items():
    if ctype == 'Not significant':
        continue
    subset = tracts_proj[tracts_proj['LISA_cluster'] == ctype]
    if not subset.empty:
        subset.plot(color=color, ax=ax, edgecolor='white', label=ctype)
        legend_handles.append(mpatches.Patch(color=color, label=ctype))

# If no significant clusters, add a placeholder legend entry
if len(legend_handles) == 0:
    legend_handles.append(mpatches.Patch(color='lightgray', label='No significant clusters'))

plt.title("Local Moran's I Cluster Map (Ozone)")
plt.legend(handles=legend_handles, loc='upper right')
ax.set_aspect('equal')
plt.show()

### Moran Scatter Plot

In [ ]:
# ------------------------------------------------------------------------------
# 11) Moran Scatter Plot
# ------------------------------------------------------------------------------

# Standardize y
y_mean = y.mean()
y_std = y.std()

if not np.isfinite(y_std) or np.isclose(y_std, 0):
    print("Warning: Standard deviation is zero or non-finite. Skipping scatter plot regression.")
    z = np.zeros_like(y)
    lag_z = np.zeros_like(y)
else:
    z = (y - y_mean) / y_std
    w_matrix = w.full()[0]   # NxN weights matrix
    lag = w_matrix @ y       # sum of neighbors
    lag_z = (lag - y_mean) / y_std

# Create color coding based on the LISA_cluster
color_map_scatter = {
    'High-High': 'red',
    'Low-Low': 'blue',
    'High-Low': 'orange',
    'Low-High': 'lightblue',
    'Not significant': 'gray'
}
colors = [color_map_scatter.get(c, 'gray') for c in tracts_gdf_largest['LISA_cluster']]

plt.figure(figsize=(6, 6))
plt.scatter(z, lag_z, c=colors, alpha=0.6)

# Reference lines at 0
plt.axhline(0, color='black', linestyle='--')
plt.axvline(0, color='black', linestyle='--')

# Optional regression line
try:
    m, b = np.polyfit(z, lag_z, 1)
    plt.plot(z, m*z + b, color='green', label="Regression line")
except np.linalg.LinAlgError as e:
    print("Regression line failed:", e)

plt.title("Moran Scatter Plot (Ozone)")
plt.xlabel("Standardized Ozone (Z)")
plt.ylabel("Standardized Spatial Lag of Ozone")
plt.legend()
plt.show()

## Results Storage and Conclusion

In [ ]:
# # Save the census tracts with aggregated pollutant data and LISA cluster info
# tracts_gdf.to_file("census_tracts_pollution.shp")

# # Save the monitors with cluster labels
# monitors_in_tracts.to_file("air_quality_monitors_clusters.shp")

# # (Optional) Save any other relevant layers, e.g., a shapefile of just significant clusters
# significant_clusters = tracts_gdf[tracts_gdf['LISA_cluster'] != 'Not significant']
# significant_clusters.to_file("significant_ozone_clusters.shp")